In [1]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

from langdetect import detect 
from collections import Counter

import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Bidirectional, Embedding, LSTM, Dense

from keras.preprocessing.sequence import pad_sequences

# Chargement du fichier de données
file_path = '/home/vatosoa/mg-smart-lingua-discover/data/corpus/sokajinteny.csv'
df = pd.read_csv(file_path)

# Colonnes 'Fehezanteny' et 'Sokajinteny' sont remplies ?
df['Fehezanteny'].fillna('', inplace=True)
df['Sokajinteny'].fillna('', inplace=True)

# Jeton spécial pour les mots inconnus
UNKNOWN_TOKEN = '<UNK>'

# Fonction de tokenisation de mots avec gestion des mots inconnus
def tokenize_words(text):
    if isinstance(text, str):
        language = detect(text)
        text = re.sub(r'[-;:()?!,.\'"\/|]', ' ', text)
        tokens = nltk.word_tokenize(text)
        tokens = [token.lower() if token.isalnum() else UNKNOWN_TOKEN for token in tokens]
        return tokens
    else:
        return []

# Appliquer la tokenisation aux phrases et créer une colonne pour les tokens
df['Fehezanteny_tokens'] = df['Fehezanteny'].apply(tokenize_words)
df['Sokajinteny_tokens'] = df['Sokajinteny'].apply(lambda x: x.split())

# Créer un encodage pour les tokens, y compris le jeton spécial
all_tokens = [token for sublist in df['Fehezanteny_tokens'] for token in sublist]
token_encoder = LabelEncoder()
token_encoder.fit(all_tokens + [UNKNOWN_TOKEN])

# Modifier les mots inconnus dans les données d'entraînement et de test
X = [[token_encoder.transform([token])[0] if token in token_encoder.classes_ else token_encoder.transform([UNKNOWN_TOKEN])[0] for token in tokens] for tokens in df['Fehezanteny_tokens'].values]
y = df['Sokajinteny_tokens'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création de l'encodeur pour les étiquettes
label_encoder = LabelEncoder()
label_encoder.fit(np.concatenate(y_train))

# Modèle LSTM
max_sequence_length = max([len(tokens) for tokens in X_train])
vocab_size = len(token_encoder.classes_)
num_labels = len(label_encoder.classes_)

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dense(num_labels, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entraînement du modèle
X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length, padding='post')
y_train_padded = pad_sequences([label_encoder.transform(labels) for labels in y_train], maxlen=max_sequence_length, padding='post')

model.fit(X_train_padded, y_train_padded, epochs=50, batch_size=32)


2024-03-02 16:19:13.285523: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-02 16:19:13.375747: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 16:19:13.375824: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 16:19:13.375882: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-02 16:19:13.389673: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-02 16:19:13.390362: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Epoch 1/50
30/30 [==============================] - 21s 408ms/step - loss: 1.1428 - accuracy: 0.7713
Epoch 2/50
30/30 [==============================] - 13s 421ms/step - loss: 0.6063 - accuracy: 0.7992
Epoch 3/50
30/30 [==============================] - 11s 373ms/step - loss: 0.5769 - accuracy: 0.7992
Epoch 4/50
30/30 [==============================] - 12s 412ms/step - loss: 0.5592 - accuracy: 0.8000
Epoch 5/50
30/30 [==============================] - 12s 412ms/step - loss: 0.5365 - accuracy: 0.8047
Epoch 6/50
30/30 [==============================] - 12s 418ms/step - loss: 0.4931 - accuracy: 0.8213
Epoch 7/50
30/30 [==============================] - 11s 367ms/step - loss: 0.4112 - accuracy: 0.8820
Epoch 8/50
30/30 [==============================] - 12s 412ms/step - loss: 0.3133 - accuracy: 0.9244
Epoch 9/50
30/30 [==============================] - 12s 417ms/step - loss: 0.2347 - accuracy: 0.9460
Epoch 10/50
30/30 [==============================] - 13s 432ms/step - loss: 0.1786 - accura

In [4]:

# Prédiction sur les données de test
X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length, padding='post')
predictions_test = model.predict(X_test_padded)

# Décodez les prédictions en étiquettes
decoded_predictions = []
for i in range(len(predictions_test)):
    decoded_sequence = []
    for j in range(len(X_test_padded[i])):
        if X_test_padded[i][j] != token_encoder.transform([UNKNOWN_TOKEN])[0]:  # Ignorez le jeton spécial
            predicted_label = np.argmax(predictions_test[i][j])
            decoded_sequence.append(label_encoder.inverse_transform([predicted_label])[0])
    decoded_predictions.append(decoded_sequence)

# Fonction pour décoder les tokens encodés en chaînes de caractères
#def decode_tokens(encoded_tokens, encoder):
#    return [encoder.inverse_transform([token])[0] if token != encoder.transform([UNKNOWN_TOKEN])[0] else UNKNOWN_TOKEN for token in encoded_tokens]

def decode_tokens(encoded_tokens, encoder):
    return [encoder.inverse_transform([token])[0] if token in encoder.classes_ else 'Unknown' for token in encoded_tokens]


# Ajouter <UNK> aux classes de l'encodeur d'étiquettes
label_encoder.classes_ = np.append(label_encoder.classes_, UNKNOWN_TOKEN)
total_test_sentences = len(X_test)

# Affichage des exemples de prédictions avec les phrases originales
print("Nombre total de phrases de test:", total_test_sentences)
for i in range(5):  # Affichage des prédictions pour les 5 premières séquences de test
    print("Exemple", i+1)
    print("Phrase d'origine:", ' '.join(decode_tokens(X_test[i], token_encoder)))
    print("Prédiction:", decode_tokens(decoded_predictions[i], label_encoder))
    print()


8/8 [==============================] - 1s 119ms/step
Nombre total de phrases de test: 234
Exemple 1
Phrase d'origine: Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown


ValueError: y contains previously unseen labels: ['Mpampitohy']